In [1]:
pip install darts

In [2]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install optuna-integration[pytorch_lightning]

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import math
from darts.models import NBEATSModel, RNNModel, TCNModel, TransformerModel
from darts.timeseries import TimeSeries as ts
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mae, rmse,mape,mse
import pickle

The StatsForecast module could not be imported. To enable support for the StatsForecastAutoARIMA, StatsForecastAutoETS and Croston models, please consider installing it.


In [5]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
)

In [6]:
df = pd.read_csv('San_Juan_Dengue_Data_Cleaned.csv')
df

,week_start_date,total_cases
0,1990-04-30,4
1,1990-05-07,5
2,1990-05-14,4
3,1990-05-21,3
4,1990-05-28,6
...,...,...
983,2009-03-26,24
984,2009-04-02,9
985,2009-04-09,21
986,2009-04-16,10


In [7]:
# Ensure the 'week_start_date' column is in datetime format
df['week_start_date'] = pd.to_datetime(df['week_start_date'])

# Set the 'week_start_date' as the index and enforce a 7-day frequency
df.set_index('week_start_date', inplace=True)

# Use asfreq with forward fill to handle any potential missing data points
df = df.asfreq('7D', method='ffill')

# Reset the index to pass it to your ts object
df.reset_index(inplace=True)

# Now pass the dataframe to your ts object
ts_dengue = ts.from_dataframe(df, time_col='week_start_date', value_cols='total_cases')


In [8]:
ts_dengue

<TimeSeries (DataArray) (week_start_date: 991, component: 1, sample: 1)>
array([[[  4.]],

       [[  5.]],

       [[  4.]],

       [[  3.]],

       [[  6.]],

       [[  2.]],

       [[  4.]],

       [[  5.]],

       [[ 10.]],

       [[  6.]],

...

       [[ 41.]],

       [[ 35.]],

       [[ 28.]],

       [[ 28.]],

       [[ 22.]],

       [[ 26.]],

       [[ 24.]],

       [[  9.]],

       [[ 21.]],

       [[ 10.]]])
Coordinates:
  * week_start_date  (week_start_date) datetime64[ns] 1990-04-30 ... 2009-04-20
  * component        (component) object 'total_cases'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [ ]:
with open('Finalscaler.pkl', 'rb') as file:
    loaded_scaler = pickle.load(file)

In [10]:
ts_dengue_scaled = loaded_scaler.fit_transform(ts_dengue)
ts_dengue_train = ts_dengue_scaled[:971]
ts_dengue_val = ts_dengue_scaled[971:981]

In [11]:
# with open('TrainedModel.pkl', 'rb') as file:
#    loaded = pickle.load(file)
# loaded

NBEATSModel(output_chunk_shift=0, generic_architecture=True, num_stacks=30, num_blocks=1, num_layers=4, layer_widths=256, expansion_coefficient_dim=5, trend_polynomial_degree=2, dropout=0.0, activation=LeakyReLU, input_chunk_length=10, output_chunk_length=5, n_epochs=100, model_name=Nbeats100Epochs, save_checkpoints=True, force_reset=True)

In [11]:
def objective(trial):
    callback = [PyTorchLightningPruningCallback(trial, monitor="train_loss")]

    # # set input_chunk_length, between 5 and 14 days
    # days_in = trial.suggest_int("days_in", 5, 14)
    # in_len = days_in * DAY_DURATION

    # # set out_len, between 1 and 13 days (it has to be strictly shorter than in_len).
    # days_out = trial.suggest_int("days_out", 1, days_in - 1)
    # out_len = days_out * DAY_DURATION

    # Other hyperparameters
    # kernel_size = trial.suggest_int("kernel_size", 5, 25)
    # num_filters = trial.suggest_int("num_filters", 5, 25)
    # weight_norm = trial.suggest_categorical("weight_norm", [False, True])
    # dilation_base = trial.suggest_int("dilation_base", 2, 4)
    # dropout = trial.suggest_float("dropout", 0.0, 0.4)
    # lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
    # include_dayofweek = trial.suggest_categorical("dayofweek", [False, True])

    num_stacks = trial.suggest_int("num_stacks", 30, 50)
    num_blocks = trial.suggest_int("num_blocks", 1, 3)
    num_layers = trial.suggest_int("num_layers", 4, 6)
    layer_widths = trial.suggest_int("layer_widths", 512, 1024)
    expansion_coefficient_dim = trial.suggest_categorical("expansion_coefficient_dim", [5, 10])
    trend_polynomial_degree = trial.suggest_int("trend_polynomial_degree", 2, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.2)
    input_chunk_length = trial.suggest_int("input_chunk_length", 15, 45)
    output_chunk_length = trial.suggest_int("output_chunk_length", 5, 15)
    activation = trial.suggest_categorical("activation", ['ReLU', 'LeakyReLU'])
    n_epochs = trial.suggest_int("n_epochs", 40, 70)
    #optimizer_kwargs = trial.suggest_categorical("optimizer_kwargs", [{'lr': 1e-3}, {'lr': 1e-2}])
    batch_size = trial.suggest_int("batch_size", 32, 64)


    model = NBEATSModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
    n_epochs=n_epochs,
    activation=activation,
    batch_size=batch_size,
    num_stacks=num_stacks,
    num_blocks=num_blocks,
    num_layers=num_layers,
    layer_widths=layer_widths,
    expansion_coefficient_dim=expansion_coefficient_dim,
    trend_polynomial_degree=trend_polynomial_degree,
    dropout=dropout,
    #optimizer_kwargs=optimizer_kwargs,
    )

    model.fit(series=ts_dengue_train, verbose=True)#, val_series=ts_dengue_val)

    # Evaluate how good it is on the validation set
    preds = model.predict(series=ts_dengue_train, n=5)
    maes = mae(ts_dengue_val, preds, n_jobs=-1, verbose=True)
    mae_val = np.mean(maes)

    return mae_val if mae_val != np.nan else float("inf")

In [12]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")


study = optuna.create_study(direction="minimize")

study.optimize(objective,n_trials=5)

# We could also have used a command as follows to limit the number of trials instead:
# study.optimize(objective, n_trials=100, callbacks=[print_callback])

# Finally, print the best value and best hyperparameters:
print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

[I 2024-10-24 16:02:34,553] A new study created in memory with name: no-name-13ff04c0-290f-4c89-ae1b-90336acaa4ee
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 72.0 M | train
-------------------------------------------------------------
72.0 M    Trainable params
4.5 K     Non-trainable params
72.0 M    Total params
288.146   Total estimated model params size (MB)
534       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=60` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=43` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2024-10-24 18:57:53,995] Trial 4 finished with value: 0.013720501125108 and parameters: {'num_stacks': 43, 'num_blocks': 1, 'num_layers': 4, 'layer_widths': 653, 'expansion_coefficient_dim': 5, 'trend_polynomial_degree': 3, 'dropout': 0.10870316683314804, 'input_chunk_length': 24, 'output_chunk_length': 5, 'activation': 'LeakyReLU', 'n_epochs': 43, 'batch_size': 63}. Best is trial 4 with value: 0.013720501125108.


Best value: 0.013720501125108, Best params: {'num_stacks': 43, 'num_blocks': 1, 'num_layers': 4, 'layer_widths': 653, 'expansion_coefficient_dim': 5, 'trend_polynomial_degree': 3, 'dropout': 0.10870316683314804, 'input_chunk_length': 24, 'output_chunk_length': 5, 'activation': 'LeakyReLU', 'n_epochs': 43, 'batch_size': 63}


In [17]:
study.best_value, study.best_trial.params

(0.013720501125108,
 {'num_stacks': 43,
  'num_blocks': 1,
  'num_layers': 4,
  'layer_widths': 653,
  'expansion_coefficient_dim': 5,
  'trend_polynomial_degree': 3,
  'dropout': 0.10870316683314804,
  'input_chunk_length': 24,
  'output_chunk_length': 5,
  'activation': 'LeakyReLU',
  'n_epochs': 43,
  'batch_size': 63})

In [18]:
import json
# Save the parameters to a JSON file
with open("best_params.json", "w") as f:
    json.dump(study.best_trial.params, f)

In [13]:
plot_optimization_history(study)

In [16]:
plot_contour(study, params=["num_layers", "num_blocks"])

In [15]:
plot_param_importances(study)